# GONZAG WestMed example

This notebook demonstrates how Gonzag can be used to:
- perform a space-time interpolation of SSH on a gridded OGCM domain onto a satellite track (provided in 2 NetCDF input files)
- select and retain N valid SSH along track segments (both for input satellite SSH and interpolated model SSH)
- preprocess these N SSH segments prior to applying a Fast Fourrier Transform (FFT)
- apply the FFT
- plot the the mean spectrum (of the N spectra)

## Loading & initializations

In [1]:
import sys
import os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

GONZAG_DIR   = '/Users/auraoupa/Work/git/gonzag_cloud'
sys.path.append(GONZAG_DIR)

import gonzag as gz

GONZAG_DATA_DIR = '/Users/auraoupa/Data/gonzag/gonzag_input'

# Satellite input data:
name_sat = 'Sentinel3A'
file_sat = GONZAG_DATA_DIR+'/SENTINEL3A_20170130-20170303.nc'
name_ssh_sat = 'sla_unfiltered'
name_time_sat='time'
period_sat=['2017-02-01','2017-02-28'] #period of satellite is bigger than model so we restrict to model period

# Model input data:
name_mod = 'eNATL60-WestMed'
file_mod = GONZAG_DATA_DIR+'/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc'
name_ssh_mod = 'sossheig'
name_lat_mod = 'gphit'
name_lon_mod = 'glamt'
name_time_mod = 'time_counter'
file_lsm_mod = file_mod; name_lsm_mod = '_FillValue' ; # we use _FillValue attribute of "nams_ssh_mod" in "file_mod"
clsm = name_lsm_mod
if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod

period_mod=['2017-02-01','2017-02-28']
l_griddist = False ; # grid is not strongly distorded





In [2]:
!ls $file_mod

/Users/auraoupa/Data/gonzag/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc


In [3]:
import xarray as xr
dsmod=xr.open_dataset(file_mod)
dssat=xr.open_dataset(file_sat)

In [4]:
dsmod

<xarray.Dataset>
Dimensions:       (y: 796, x: 868, time_counter: 672)
Coordinates:
    glamt         (y, x) float64 ...
    gphit         (y, x) float64 ...
  * time_counter  (time_counter) datetime64[ns] 2017-02-01T00:30:00 ... 2017-...
Dimensions without coordinates: y, x
Data variables:
    sossheig      (time_counter, y, x) float32 ...
Attributes:
    About:    Created with SOSIE interpolation environement => https://github...
    history:  Wed Mar 31 10:18:30 2021: ncks -4 -L 7 --cnk_dmn x,868 --cnk_dm...
    NCO:      netCDF Operators version 4.9.1 (Homepage = http://nco.sf.net, C...

In [5]:
dssat

<xarray.Dataset>
Dimensions:         (time: 1599082)
Coordinates:
  * time            (time) datetime64[ns] 2017-01-29T23:44:39.027517696 ... 2...
    longitude       (time) float64 ...
    latitude        (time) float64 ...
Data variables:
    cycle           (time) int16 ...
    track           (time) int16 ...
    sla_unfiltered  (time) float64 ...
Attributes: (12/45)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea surface height measured by altimeter...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    time_coverage_end:               2017-01-30T23:06:37Z
    time_coverage_resolution:        P1S
    time_coverage_start:             2017-01-29T23:44:39Z
    title:                           DT Sentinel-3A Global Ocean Along track ...
    NCO:                             netCDF Operators version 4.9.1 (Homepage...
    history:                         Fri Apr  2 12:02:02 2021: ncwa -O -a rec...

## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [6]:
ModelGrid = gz.ModGrid( dsmod, period_mod, name_lon_mod, name_lat_mod , name_time_mod, dsmod, clsm, distorded_grid=False )



 *** what we use to define model land-sea mask:
    => "_FillValue@sossheig" in dataset 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (796, 868)
     * horizontal resolution:  0.016753984549605475  degrees or  1.8615352233066644  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [-180:180] frame...
     * lon_min, lon_max =  -5.49 9.51
     * lat_min, lat_max =  35.08 44.43
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  648
       ==> time record dates: 2017-02-01 to 2017-02-28, included



## Create object `SatelliteTrack` containing all the satellite track (aka _target_) info

In [7]:
SatelliteTrack = gz.SatTrack( dssat, period_sat, name_time_sat, name_ssh_sat, domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  2168
       ==> time record indices: 94780 to 1365984, included

       separated in 42 tracks


## Build the bilinear mapping & Perform the space-time interpolation

In [8]:
if not os.path.exists('../results/results_'+name_sat+'-'+name_mod):
        os.makedirs('../results/results_'+name_sat+'-'+name_mod)

In [11]:
import numpy as nmp
def process_one_track(track):
    tt = "{:02d}".format(track)
    Solution0 = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track )
    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]
    from gonzag.config import rmissval
    iw = gz.io.SaveTimeSeries( Solution0.time, \
                             nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
                                         Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] ), \
                             vvar, '../results_bis/results_'+name_sat+'-'+name_mod+'/result_'+str(tt)+'.nc', time_units='',\
                             vunits=vunits, vlnm=vlongN, missing_val=rmissval )
    return Solution0


In [13]:
process_one_track(0)


 *** "../results_bis/results_Sentinel3A-eNATL60-WestMed/result_00.nc" successfully written!

